In [1]:
%run init_notebook.py

In [2]:
import eurostat
from src.utils import load_pd_df, get_dt_index, save_pd_df

In [6]:
# interest rates

In [7]:
df = pd.read_csv(os.path.join(DATA_DIR, 'ecb_interest_rates.csv'), low_memory=False)
df = df[df['REF_AREA'] == 'DE'].dropna(axis=1)
df['date'] = df['TIME_PERIOD'].apply(lambda x: datetime.strptime(x, "%Y-%m"))
df = df[df['TITLE'] == list(set(df['TITLE']))[1]]
df.set_index('date', inplace=True)

mir = df[['OBS_VALUE']].rename(columns={'OBS_VALUE': 'mir'}) / 100

In [17]:
save_pd_df(mir, 'mir.csv')

In [4]:
# eurostat

In [5]:
codes = eurostat.get_toc_df()

In [6]:
[i for i in codes['title'] if 'consum'.lower() in i.lower()];

In [8]:
filt = [
    'HICP - energy', 
    'HICP - food, alcohol and tobacco',
    'Gross domestic product (GDP) at current market prices by metropolitan regions',
    'Gross domestic product (GDP) at market prices - quarterly data',
    'HICP - inflation rate',
    'HICP - all items excluding energy and seasonal food',
    'HICP - energy',
    'HICP - monthly data (index)',
    'Final consumption expenditure',
]
filt = [i in filt for i in codes['title']]

In [9]:
codes[filt];

In [10]:
# overall inflation

In [11]:
data = eurostat.get_data('med_ec2') #, filter_pars={'geo': ['DE','FR']})

In [11]:
df = pd.DataFrame(data[1:], columns=data[0]).transpose()
df = df.loc[:, (df.loc['unit'] == 'I05') & (df.loc['coicop'] == 'CP00')]
df.columns = [f'cpi_{i}' for i in df.loc['geo\TIME_PERIOD'].values]

cpi = df.iloc[5:].copy().dropna().astype(float)
cpi.index = [datetime(int(i[:4]), int(i[5:7]), 1) for i in cpi.index]

In [12]:
cpi['pi_de'] = np.log(cpi['cpi_DE']).diff()

In [13]:
save_pd_df(cpi, 'cpi.csv')

In [14]:
# food tobaco inflation

In [15]:
data = eurostat.get_data('TEICP260', filter_pars={'geo': ['DE']})
data = np.array(data)

In [16]:
df = pd.DataFrame(data=data[1:], columns=data[0])
df

,freq,coicop,unit,geo\TIME_PERIOD,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01
0,M,FOOD,I15,DE,139.5,141.3,141.1,141.1,141.2,141.2,141.0,141.5,141.6,142.4,142.4,None
1,M,FOOD,PCH_M1,DE,2.0,1.3,-0.1,0.0,0.1,0.0,-0.1,0.4,0.1,0.6,0.0,0.9
2,M,FOOD,PCH_M12,DE,18.0,18.6,15.5,13.6,12.6,10.7,9.2,8.1,7.1,6.6,5.8,5.1


In [3]:
# pi prognosis

In [4]:
df = get_dt_index(load_pd_df('ecb_prognosis_hicp_dec_2021.csv'), dt_index_col="DATE").iloc[:, 1:]
df.index.name = 'date'
df.columns = ["infl_forecast"]

In [5]:
df1 = load_pd_df('ecb_prognosis_hicp_jun_2021.csv')
df1['date'] = df1.DATE.apply(lambda x: datetime(int(x[:4]), 6, 30))
df1 = df1.drop('DATE', axis=1).set_index('date').iloc[:, 1:]
df1.columns = ["infl_forecast"]

df = pd.concat([df, df1], axis=0).sort_index()

In [19]:
save_pd_df(df, 'infl_prognonsis.csv')

# hhp

In [65]:
dfs, metadata = [], []
for file in os.listdir(os.path.join(DATA_DIR, 'hhs')):
    print(file)
    try:
        dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file))
        dfs.append(dta)
        metadata.append(metadata)
    except Exception as e:
        print(e)
        continue

bophh_suf_202304_v01_wave01.dta
bophh_suf_202304_v01_wave02.dta
bophh_suf_202304_v01_wave03.dta
bophh_suf_202304_v01_wave04.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_23700\1408175798.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file))


bophh_suf_202304_v01_wave05.dta
bophh_suf_202304_v01_wave06.dta
bophh_suf_202304_v01_wave07.dta
bophh_suf_202304_v01_wave08.dta
bophh_suf_202304_v01_wave09.dta
bophh_suf_202304_v01_wave10.dta
bophh_suf_202304_v01_wave11.dta
bophh_suf_202304_v01_wave12.dta
bophh_suf_202304_v01_wave13.dta
bophh_suf_202304_v01_wave14.dta
bophh_suf_202304_v01_wave15.dta
bophh_suf_202304_v01_wave16.dta
bophh_suf_202304_v01_wave17.dta
bophh_suf_202304_v01_wave18.dta
bophh_suf_202304_v01_wave19.dta
bophh_suf_202304_v01_wave20.dta
bophh_suf_202304_v01_wave21.dta
bophh_suf_202304_v01_wave22.dta
bophh_suf_202304_v01_wave23.dta
bophh_suf_202304_v01_wave24.dta
bophh_suf_202304_v01_wave25.dta
bophh_suf_202304_v01_wave26.dta
bophh_suf_202304_v01_wave27.dta
'utf-8' codec can't decode byte 0xfc in position 8: invalid start byte
bophh_suf_202304_v01_wave28.dta
'utf-8' codec can't decode byte 0x80 in position 5: invalid start byte
bophh_suf_202304_v01_wave29.dta
bophh_suf_202304_v01_wave30.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_23700\1408175798.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file))


bophh_suf_202304_v01_wave31.dta


...

bophh_suf_202304_v01_wave39.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_23700\1408175798.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file))


bophh_suf_202304_v01_wave40.dta
bophh_suf_202304_v01_wave41.dta


C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_23700\1408175798.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  dta = pd.read_stata(os.path.join(DATA_DIR, 'hhs', file))


bophh_suf_202304_v01_wave42.dta
bophh_suf_202304_v01_wave43.dta
bophh_suf_202304_v01_wave44.dta
bophh_suf_202304_v01_wave45.dta


In [67]:
data = pd.concat(dfs, axis=0)

In [68]:
data['riskaversion'] = data.riskaversion.cat.codes.replace({-1:np.nan})
data['ambig_lot'] = data.ambig_lot.cat.codes.replace({-1:np.nan})

In [72]:
dict_cols_parse = {
    'id': float,
#     'date': 'datetime64[ns]',
    'year': int,
    'month': int,
    'day': int,
    'riskaversion': str,
    'expint_sav': float,
    'devinfpoint
    ': float,
    'inflexppoint': float    
}
dict_cols_parse.update({f'infexprob_{i}': str for i in list(map(chr, range(97, 107)))})
dict_cols_parse.update({f'incexp_{i}': str for i in list(map(chr, range(97, 107)))})

_ = [
        "don't know",
        'no answer',
        'does not apply',
    ]
dict_parse_nan = dict(zip(_, list([np.nan]) * len(_)))

In [73]:
df = data[list(dict_cols_parse.keys())]

In [75]:
del data, dfs

In [76]:
df['date'] = df[['year', 'month', 'day']].apply(lambda x: datetime(*x.astype(int)), axis=1).rename('date').values
df = df.drop(['year', 'month', 'day'], axis=1)
df['inflexppoint'] = df['inflexppoint'].apply(lambda x: dict_parse_nan[x] if x in dict_parse_nan.keys() else x)

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_23700\3676133331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df[['year', 'month', 'day']].apply(lambda x: datetime(*x.astype(int)), axis=1).rename('date').values


In [79]:
df['devinfpoint'] = df['devinfpoint'].apply(lambda x: dict_parse_nan[x] if x in dict_parse_nan.keys() else x)

In [86]:
df = df.replace(dict_parse_nan)

C:\Users\LukasGrahl\AppData\Local\Temp\ipykernel_23700\2627956439.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(dict_parse_nan)


In [87]:
df = df.astype({k: v for k, v in dict_cols_parse.items() if k in df.columns})

In [89]:
df = df.loc[df.inflexppoint.dropna().index]
df.sort_values('date', inplace=True)

df['y_m'] = df.date.dt.to_period('M').astype(str)

In [90]:
df = df.replace({'nan': np.nan})

In [91]:
df['date_forecast'] = df.date.apply(lambda x: datetime(x.year + 1, x.month, x.day))
df = df.rename(columns={'date': 'date_recorded'})

In [93]:
df.columns

Index(['id', 'riskaversion', 'expint_sav', 'devinfpoint', 'inflexppoint',
       'infexprob_a', 'infexprob_b', 'infexprob_c', 'infexprob_d',
       'infexprob_e', 'infexprob_f', 'infexprob_g', 'infexprob_h',
       'infexprob_i', 'infexprob_j', 'incexp_a', 'incexp_b', 'incexp_c',
       'incexp_d', 'incexp_e', 'incexp_f', 'incexp_g', 'incexp_h', 'incexp_i',
       'incexp_j', 'date_recorded', 'y_m', 'date_forecast'],
      dtype='object')

In [92]:
df.to_feather(os.path.join(DATA_DIR, 'hhp.feather'))